In [3]:
!pip install opensearch-py -U -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [67]:
!docker build -t task-store:latest . --quiet

sha256:b3f920e1f23963e325261b15e172c739bfcf39df055e785b6c971303ab9391fe


In [54]:
!docker run -d -p 8080:8080 -e AWS_DEFAULT_REGION="us-west-2" -e AWS_REGION="us-west-2" --name test-task-store task-store:latest --host 0.0.0.0 --port 8080 --workers 3 --aoss-endpoint "os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com" --aoss-index "task-store-index"

8aa79c85151942973051bb74831d537a31443e23ad78459df3bbe5088a2c504d


In [40]:
!docker logs test-task-store

INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started parent process [1]
INFO:     Started server process [9]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [8]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [10]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [59]:
!docker kill test-task-store && docker rm -f test-task-store

test-task-store
test-task-store


In [38]:
!docker rm -f test-task-store

test-task-store


In [36]:
!docker context ls

NAME                             DESCRIPTION                               DOCKER ENDPOINT                                       ERROR
default                          Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                           
m5.large_i-0e2fa115a75372436 *                                             tcp://ip-10-3-1-173.us-west-2.compute.internal:1111   


In [24]:
host_ip = "ip-10-3-1-173.us-west-2.compute.internal"

In [41]:
%%bash -s $host_ip
curl ${1}:8080/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22  100    22    0     0   1888      0 --:--:-- --:--:-- --:--:--  2000


{"Health_Check":"200"}

In [16]:
import requests
import json

host_ip = "internal-codenator-899847730.us-west-2.elb.amazonaws.com"
port = 8010
task_desc = "Write Hello world code"
model_family = "bedrock"
model_name = "amazon.titan-embed-text-v1"
url = f"http://{host_ip}:{port}/invoke"
data = {
    "body": {"prompt": task_desc}, 
    "model_family": model_family, 
    "model_name": model_name
    
}
embedding = json.loads(requests.post(url=url, data=json.dumps(data)).text)["embedding"]

# host_ip = "ip-10-3-1-173.us-west-2.compute.internal"
# port = 8080

# url = f"http://{host_ip}:{port}/save_task"
# data = {
#     "embedding": embedding,
#     "script": "print(\"Hello World!\")",
#     "task_desc": task_desc,
#     "language": "python"
# }
# print(requests.post(url=url, data=json.dumps(data)).text)

In [94]:
host_ip = "internal-codenator-899847730.us-west-2.elb.amazonaws.com"
port = 8014
url = f"http://{host_ip}:{port}/aoss/list_tasks"
print(requests.get(url=url).json())

{}


In [93]:
url = f"http://{host_ip}:{port}/load_task"
data = {
    "embedding": embedding,
    "threshold": 0.5
}
print(requests.post(url=url, data=json.dumps(data)).text)

[{"embedding":[0.43554688,0.09472656,0.13574219,0.21191406,-0.032470703,0.609375,0.029296875,-0.00037384033,0.14160156,-0.49023438,-0.0050354004,-0.53125,0.2890625,0.47070312,-0.13476562,0.033691406,-0.37890625,0.7890625,-0.578125,-0.12890625,-0.5859375,0.41210938,0.014526367,0.25,-0.18554688,0.040039062,0.16015625,-0.8515625,0.53515625,-0.04638672,-0.033935547,0.38085938,0.2734375,-0.35546875,-0.375,-0.62109375,0.51171875,-0.4140625,0.21191406,0.19433594,0.00793457,0.79296875,0.10644531,-0.59765625,-0.41210938,0.095214844,0.018554688,-0.70703125,0.7421875,0.69921875,0.29101562,0.19628906,0.039794922,-0.123046875,0.1953125,-0.06689453,0.453125,-0.4296875,0.62109375,-0.43359375,-0.515625,0.08154297,-0.2734375,0.40625,-0.23339844,0.1796875,-0.26367188,-0.3359375,-0.68359375,0.9921875,-0.06201172,-0.06591797,0.45898438,0.55859375,0.35546875,-0.032714844,0.125,0.27539062,-0.039794922,0.7109375,0.049316406,0.26757812,0.111816406,0.12109375,-0.546875,0.21972656,-0.55078125,-0.20605469,0.0001

In [4]:
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection
import boto3

# def push_to_aws():
host = "os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com"
credentials = boto3.Session().get_credentials()
region = "us-west-2" 

# Sign the credentials
auth = AWSV4SignerAuth(credentials, region, service="aoss")

# The OpenSearch client
client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

In [54]:
index_name = "task-store-index"
index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 512
    }
  },
  "mappings": {
    "properties": {
      "embedding": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "name": "hnsw",
          "engine": "nmslib",
          "parameters": {},
          "space_type": "cosinesimil"
        }
      }
    }
  }
}
response = client.indices.create(index_name, body=index_body)
print(response)

In [60]:
client.indices.get_alias(index="*")

{'.opensearch_dashboards_1': {'aliases': {'.opensearch_dashboards': {}}},
 'task-store-index': {'aliases': {}}}

In [116]:
document = {
  "embedding": [0] * 1536,
  "task_desc": "Bennett Miller",
  "code": "2011"
}

response = client.index(
    index = "task-store-index",
    body = document
)
print(response)

{'_index': 'task-store-index', '_id': '1%3A0%3AekIuBowBqeZyZXfS5mmF', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}


In [13]:
client.delete(index="task-store-index", id="1%3A0%3AV7MpEYwBp55xbD10iHyx")

{'_index': 'task-store-index',
 '_id': '1%3A0%3AV7MpEYwBp55xbD10iHyx',
 '_version': 2,
 'result': 'deleted',
 '_shards': {'total': 0, 'successful': 0, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 0}

In [76]:
print(response)

{'_index': 'task-store-index', '_id': '1%3A0%3AeUL1BYwBqeZyZXfSyGnW', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}


In [17]:
query = {
  "size": 1,
  "query": {
      "knn": {
          "embedding": {
              "vector": embedding,
              "k": 1
          }
      }
  }
}

response = client.search(
    body = query,
    index = "task-store-index"
)
stored_emb = response["hits"]["hits"][0]["_source"]["embedding"]

In [19]:
import math
from numpy import dot
from numpy.linalg import norm

def calculate_cosine_similarity(v1, v2):
    similarity = dot(v1, v2)/(norm(v1)*norm(v2))
    return 1- similarity


def calculate_distance(v1, v2):
    distance = math.dist(v1, v2)
    return distance

print(calculate_distance(stored_emb, embedding))
print(calculate_cosine_similarity(stored_emb, embedding))

16.027821894805967
0.22985853815102042


In [20]:
query = {
  "size": 100,
  "query": {
      "match_all": {
      }
  }
}

response = client.search(
    body = query,
    index = "task-store-index"
)
print(response)

{'took': 69, 'timed_out': False, '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'task-store-index', '_id': '1%3A0%3AfkIXEYwBqeZyZXfS12ne', '_score': 1.0, '_source': {'embedding': [0.43359375, 0.10595703, 0.15527344, 0.22851562, -0.024291992, 0.59375, 0.018798828, -0.00037384033, 0.16308594, -0.5, -0.03149414, -0.515625, 0.28710938, 0.47460938, -0.12695312, 0.033691406, -0.39257812, 0.828125, -0.6171875, -0.15917969, -0.59375, 0.40429688, 0.003036499, 0.25585938, -0.22167969, 0.047607422, 0.13183594, -0.82421875, 0.53515625, -0.018310547, -0.12060547, 0.37890625, 0.30078125, -0.40234375, -0.35546875, -0.58203125, 0.51953125, -0.41796875, 0.18261719, 0.20117188, 0.015380859, 0.796875, 0.09716797, -0.63671875, -0.37890625, 0.09326172, 0.013244629, -0.71875, 0.7578125, 0.71484375, 0.28710938, 0.21972656, 0.028442383, -0.087890625, 0.20898438, -0.06738281, 0.45507812, -0.45703125, 0

In [90]:
response["hits"]

{'total': {'value': 1, 'relation': 'eq'},
 'max_score': 0.5,
 'hits': [{'_index': 'task-store-index',
   '_id': '1%3A0%3AeEL1BYwBqeZyZXfSjWl4',
   '_score': 0.5,
   '_source': {'embedding': [0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0

In [52]:
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection
from opensearch_dsl import Search
import boto3

# def push_to_aws():
host = f"os55okuuw7koc3rgpi13.us-west-2.aoss.amazonaws.com"
credentials = boto3.Session().get_credentials()
region = "us-west-2" 

# Sign the credentials
auth = AWSV4SignerAuth(credentials, region, service="aoss")

# The OpenSearch client
client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

my_index = "task-store-index"

try:
    response = client.indices.create(my_index)
    print("\nCreating index:")
    print(response)
except Exception as e:
    print(e)

#     files = glob.glob("/home/martin/json_directory/*.json")

#     for file in files:
#         with open(file, "r") as f:
#             print(f"Processing {file}")
#             data = payload_constructor(data=json.load(f), action={"index": {}})

#             response = client.bulk(body=data, index=my_index)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'task-store-index'}
